In [72]:
import sklearn.mixture
import pickle
import numpy as np
from sklearn.preprocessing import normalize

In [73]:
AMFCC_path='/home/ubuntu/genderRecognition/AMFCC.pk'
GMM_Path='/home/ubuntu/genderRecognition/GMM.pk'
numc_comp=64

In [74]:
male_model= sklearn.mixture.GaussianMixture(n_components=numc_comp, 
                                       covariance_type='diag')
female_model=sklearn.mixture.GaussianMixture(n_components=numc_comp, 
                                       covariance_type='diag')

In [4]:
f=open(AMFCC_path,'rb')
[train_set_AMFCC,valid_set_AMFCC,test_set_AMFCC,train_set_y,valid_set_y,test_set_y]=pickle.load(f)
f.close()

In [75]:
train_female_feats=np.concatenate([(train_set_AMFCC[i]-np.mean(train_set_AMFCC[i],axis=0))/np.linalg.norm(train_set_AMFCC[i],axis=0) for i in range(len(train_set_AMFCC))[:len(train_set_AMFCC)/2]],axis=0)
train_male_feats=np.concatenate([(train_set_AMFCC[i]-np.mean(train_set_AMFCC[i],axis=0))/np.linalg.norm(train_set_AMFCC[i],axis=0) for i in range(len(train_set_AMFCC))[len(train_set_AMFCC)/2:]],axis=0)

In [76]:
male_model.fit(train_male_feats)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=64, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [77]:
female_model.fit(train_female_feats)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=64, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [78]:
f=open(GMM_Path,'wb')
pickle.dump((male_model,female_model), f)
f.close()

In [131]:
from base import GenderPredict
prediction=[]
count=0
for i in range(len(test_set_y)):
    count+=1
    if count % int(len(test_set_y)*0.1)==0:
        print('done %s%%'%int(count*10/(len(test_set_y)*0.1)))
    prediction.append(GenderPredict(test_set_AMFCC[i],male_model,female_model))
comparison=[prediction[i]==test_set_y[i] for i in range(len(test_set_AMFCC))]
accuracy=1.0*sum(comparison)/len(test_set_y) 
print('Accuracy is %f%%'%(accuracy*100))

done 10%
done 20%
done 30%
done 40%
done 50%
done 60%
done 70%
done 80%
done 90%
done 100%
Accuracy is 79.750000%
